In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(action = 'ignore')
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)

In [3]:
vod_08 = pd.read_csv('../data/데이터스쿨3차_2308월/데이터스쿨_3차_VOD_2308.csv', encoding = 'cp949', sep = '\t')
vod_09 = pd.read_csv('../data/데이터스쿨3차_2309월/데이터스쿨_3차_VOD_2309.csv', encoding = 'cp949', sep = '\t')
vod_id = pd.read_csv('../data/전처리data/vod_id_origin.csv', encoding = 'cp949')

In [5]:
# 8,9월 데이터 합치기
vod_89 = pd.concat([vod_08, vod_09], ignore_index=True)
vod_89.head(2)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230816205227


In [10]:
from pandas._libs.tslibs.offsets import Hour
def preprocessing(data):
  df = data.copy()

  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  # 런타임 0분인 것 제거
  df = df[df['disp_rtm'] != 0]

  # 무삭제 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'무삭제판|무삭제', '', regex = True)

  # 예고편 제거
  df = df[~df['asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]

  # 예약구매, 사전구매 제거
  df = df[~df['asset_nm'].str.contains('예약구매|사전구매')]

  # 00회가 포함된 행은 런타임이 0또는 1이므로 제거
  df = df[~df['asset_nm'].str.contains(r'\b00회\b')]

  ## 괄호와 그 안의 내용 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\…', '', regex=True) # … 기호 제거
  df['asset_nm'] = df['asset_nm'].str.rstrip('.')
  df['asset_nm'] = df['asset_nm'].str.replace('-', " ")
  df['asset_nm'] = df['asset_nm'].str.strip()

  # use_tms 분 단위로 변경
  df['use_tms'] = round(df['use_tms'] / 60, 1)

  # 셋탑번호가 66056000인 것 삭제 - content 에서 이상치 아이디로 판별
  df = df[df['subsr'] != 66056000]

  return df

In [11]:
dff = preprocessing(vod_89)
data = dff.copy()
data.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737


In [12]:
# df rename 생성
data['rename'] = data['asset_nm'].copy()
data['rename'] = data['rename'].str.replace(r'\d+회$', '', regex=True)
data['rename'] = data['rename'].str.replace(r'\d+회\.', '', regex=True)
data['rename'] = data['rename'].str.replace(r'\d+화$', '', regex=True)
data['rename'] = data['rename'].str.strip()
data['rename'] = data['rename'].str.rstrip('.')

data['category'] = data['ct_cl'].apply(lambda x : x if x in ['영화', '키즈'] else 'TV프로그램')
data['rename'] = data['rename'].apply(lambda x : x.replace(' ', '') if isinstance(x, str) else x)

data.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,rename,category
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507,그것이알고싶다,TV프로그램
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227,그것이알고싶다,TV프로그램
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727,그것이알고싶다,TV프로그램
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609,그것이알고싶다,TV프로그램
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737,꼬리에꼬리를무는그날이야기,TV프로그램


In [13]:
# id 병합
id = vod_id[['rename', 'Category', 'vod_id']]
id.columns = ['rename', 'category', 'vod_id']
id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
4710,베놈2:렛데어비카니지,영화,4710
4711,성월동화,영화,4711
4712,다큐인사이트코드블루,TV프로그램,4712
4713,캠프제로베이스원,TV프로그램,4713


In [14]:
df = data.merge(id, on = ['rename', 'category'], how = 'left')
df.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,rename,category,vod_id
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507,그것이알고싶다,TV프로그램,297
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227,그것이알고싶다,TV프로그램,297
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727,그것이알고싶다,TV프로그램,297
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609,그것이알고싶다,TV프로그램,297
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737,꼬리에꼬리를무는그날이야기,TV프로그램,8


In [15]:
df[df['vod_id'].isna()]

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,rename,category,vod_id


In [16]:
# 프로그램 회차별 정보
df1 = df[['asset_nm', 'ct_cl', 'genre_of_ct_cl', 'ACTR_DISP', 'disp_rtm', 'rename', 'vod_id']].drop_duplicates().reset_index(drop = True)
df1.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,rename,vod_id
0,그것이알고싶다 1361회,TV 시사/교양,기타,김상중,80,그것이알고싶다,297
1,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,"장도연,장현성,장성규",73,꼬리에꼬리를무는그날이야기,8
2,꼬리에꼬리를무는그날이야기 61회,TV 시사/교양,기타,"장도연,장현성,장성규",73,꼬리에꼬리를무는그날이야기,8
3,인간극장 3338회,TV 시사/교양,기타,명선 스님,32,인간극장,542
4,꼬리에꼬리를무는그날이야기 89회,TV 시사/교양,기타,"장도연,장현성,장성규",78,꼬리에꼬리를무는그날이야기,8


In [17]:
# 유저마다 각 회차를 본 시간을 더함
df2 = pd.DataFrame(df.groupby(['subsr','asset_nm', 'vod_id'])['use_tms'].sum()).reset_index()
df2

,subsr,asset_nm,vod_id,use_tms
0,59879000,소방서 옆 경찰서 05회,2092,2.4
1,59879000,소방서 옆 경찰서 06회,2092,22.1
2,59879000,신성한 이혼 01회,1190,12.6
3,59879000,신성한 이혼 02회,1190,16.8
4,59879000,신성한 이혼 03회,1190,4.3
...,...,...,...,...
5661,67148000,타요의 씽씽극장 동요2 10회,880,2.0
5662,67148000,타요의 씽씽극장 동요2 11회,880,0.5
5663,67154000,스트릿 우먼 파이터 2 04회,476,1.2
5664,67161000,스파이 코드명 포춘,3824,0.2


In [18]:
# 회차별 정보와 유저 시청정보를 합친 데이터프레임
df3 = df2.merge(df1, on = ['asset_nm', 'vod_id'], how = 'left')
df3 = df3[(df3['disp_rtm'] != 0) & (df3['use_tms'] != 0)]
df3.head()

,subsr,asset_nm,vod_id,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,rename
0,59879000,소방서 옆 경찰서 05회,2092,2.4,TV드라마,기타,"김래원,손호준,공승연",69,소방서옆경찰서
1,59879000,소방서 옆 경찰서 06회,2092,22.1,TV드라마,기타,"김래원,손호준,공승연",59,소방서옆경찰서
2,59879000,신성한 이혼 01회,1190,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64,신성한이혼
3,59879000,신성한 이혼 02회,1190,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62,신성한이혼
4,59879000,신성한 이혼 03회,1190,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63,신성한이혼


In [19]:
# 시청시간으로 유저가 이 회차를 시청했는지 여부를 결정
## 시청시간을 런타임으로 나눠 0.1 이 넘으면 시청한 것으로 간주
df3['watched'] = df3['use_tms'] / df3['disp_rtm']
df3['watched'] = df3['watched'].apply(lambda x : 1 if x >= 0.2 else 0)
df3.head()

,subsr,asset_nm,vod_id,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,rename,watched
0,59879000,소방서 옆 경찰서 05회,2092,2.4,TV드라마,기타,"김래원,손호준,공승연",69,소방서옆경찰서,0
1,59879000,소방서 옆 경찰서 06회,2092,22.1,TV드라마,기타,"김래원,손호준,공승연",59,소방서옆경찰서,1
2,59879000,신성한 이혼 01회,1190,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64,신성한이혼,0
3,59879000,신성한 이혼 02회,1190,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62,신성한이혼,1
4,59879000,신성한 이혼 03회,1190,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63,신성한이혼,0


In [20]:
df1

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,rename,vod_id
0,그것이알고싶다 1361회,TV 시사/교양,기타,김상중,80,그것이알고싶다,297
1,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,"장도연,장현성,장성규",73,꼬리에꼬리를무는그날이야기,8
2,꼬리에꼬리를무는그날이야기 61회,TV 시사/교양,기타,"장도연,장현성,장성규",73,꼬리에꼬리를무는그날이야기,8
3,인간극장 3338회,TV 시사/교양,기타,명선 스님,32,인간극장,542
4,꼬리에꼬리를무는그날이야기 89회,TV 시사/교양,기타,"장도연,장현성,장성규",78,꼬리에꼬리를무는그날이야기,8
...,...,...,...,...,...,...,...
3850,엄마가 화났다,키즈,학습,-,8,엄마가화났다,3675
3851,핑크퐁 자동차 동화 01회,키즈,학습,핑크퐁,4,핑크퐁자동차동화,464
3852,간질간질,키즈,학습,-,6,간질간질,3674
3853,월간 아기상어 01회,키즈,학습,아기상어,1,월간아기상어,406


In [22]:
# 회차 제거
df11 = df1.copy()

df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+화$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.strip()
df11['asset_nm'] = df11['asset_nm'].str.rstrip('.')
df11.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,rename,vod_id
0,그것이알고싶다,TV 시사/교양,기타,김상중,80,그것이알고싶다,297
1,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73,꼬리에꼬리를무는그날이야기,8
2,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73,꼬리에꼬리를무는그날이야기,8
3,인간극장,TV 시사/교양,기타,명선 스님,32,인간극장,542
4,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",78,꼬리에꼬리를무는그날이야기,8


In [23]:
df11[df11['asset_nm'].str.contains('\.')]

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,rename,vod_id
1811,가디언즈 오브 갤럭시 VOL.3,영화,SF/환타지,"크리스 프랫,조 샐다나,데이브 바티스타,빈 디젤,브래들리 쿠퍼",150,가디언즈오브갤럭시VOL.3,2350
3750,좋.댓.구,영화,코미디,오태경,80,좋.댓.구,3365


In [24]:
# 유저 시청 정보의 회차 제거한 변수 추가
df3['series_nm'] = df3['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df3['series_nm'] = df3['series_nm'].str.replace(r'\d+회\.', '', regex=True)
df3['series_nm'] = df3['series_nm'].str.replace(r'\d+화$', '', regex=True)
df3['series_nm'] = df3['series_nm'].str.strip()
df3['series_nm'] = df3['series_nm'].str.rstrip('.')
df3.head()

,subsr,asset_nm,vod_id,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,rename,watched,series_nm
0,59879000,소방서 옆 경찰서 05회,2092,2.4,TV드라마,기타,"김래원,손호준,공승연",69,소방서옆경찰서,0,소방서 옆 경찰서
1,59879000,소방서 옆 경찰서 06회,2092,22.1,TV드라마,기타,"김래원,손호준,공승연",59,소방서옆경찰서,1,소방서 옆 경찰서
2,59879000,신성한 이혼 01회,1190,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64,신성한이혼,0,신성한 이혼
3,59879000,신성한 이혼 02회,1190,16.8,TV드라마,기타,"조승우,한혜진,김성균,정문성",62,신성한이혼,1,신성한 이혼
4,59879000,신성한 이혼 03회,1190,4.3,TV드라마,기타,"조승우,한혜진,김성균,정문성",63,신성한이혼,0,신성한 이혼


In [27]:
df33 = df3[['subsr', 'series_nm', 'vod_id', 'ct_cl',	'genre_of_ct_cl', 'ACTR_DISP',	'watched', 'rename']].copy()
df33.head()

,subsr,series_nm,vod_id,ct_cl,genre_of_ct_cl,ACTR_DISP,watched,rename
0,59879000,소방서 옆 경찰서,2092,TV드라마,기타,"김래원,손호준,공승연",0,소방서옆경찰서
1,59879000,소방서 옆 경찰서,2092,TV드라마,기타,"김래원,손호준,공승연",1,소방서옆경찰서
2,59879000,신성한 이혼,1190,TV드라마,기타,"조승우,한혜진,김성균,정문성",0,신성한이혼
3,59879000,신성한 이혼,1190,TV드라마,기타,"조승우,한혜진,김성균,정문성",1,신성한이혼
4,59879000,신성한 이혼,1190,TV드라마,기타,"조승우,한혜진,김성균,정문성",0,신성한이혼


In [28]:
# 유저의 시리즈별 시청 횟수
df4 = df33.groupby(['subsr', 'series_nm', 'vod_id'])['watched'].sum().reset_index()
df4 = df4[df4['watched']!= 0]
df4

,subsr,series_nm,vod_id,watched
0,59879000,소방서 옆 경찰서,2092,1
1,59879000,신성한 이혼,1190,3
2,59895000,금이야 옥이야,784,1
3,59900000,2022 역사저널 그날,572,1
4,59900000,그것이알고싶다,297,3
...,...,...,...,...
2066,67140000,경남 통영 2부,4396,1
2067,67140000,밀수,2865,1
2068,67140000,잠자는 숲속의 공주,689,1
2070,67148000,타요의 씽씽극장 동요2,880,11


In [29]:
# 8~9월 시리즈별 시청된 총 횟수
df5 = pd.DataFrame(df11[['asset_nm','vod_id']].value_counts().reset_index())
df5.columns = ['series_nm', 'vod_id', 'watched_all']
df5

,series_nm,vod_id,watched_all
0,금이야 옥이야,784,83
1,연희공략: 건륭황제의여인,858,63
2,TV소설 은희,315,63
3,인간극장,542,55
4,런닝맨,296,54
...,...,...,...
1018,봉신방,2423,1
1019,봉신연의,4117,1
1020,부산,1457,1
1021,북 오브 러브,1310,1


In [30]:
df6 = df4.merge(df5, on = ['series_nm', 'vod_id'] , how = 'left')
df6

,subsr,series_nm,vod_id,watched,watched_all
0,59879000,소방서 옆 경찰서,2092,1,11
1,59879000,신성한 이혼,1190,3,12
2,59895000,금이야 옥이야,784,1,83
3,59900000,2022 역사저널 그날,572,1,3
4,59900000,그것이알고싶다,297,3,21
...,...,...,...,...,...
1495,67140000,경남 통영 2부,4396,1,1
1496,67140000,밀수,2865,1,2
1497,67140000,잠자는 숲속의 공주,689,1,1
1498,67148000,타요의 씽씽극장 동요2,880,11,13


In [31]:
df7 = df6.groupby(['subsr'])['watched'].sum().reset_index()
df7.columns = ['subsr', 'watched_cnt']
df7

,subsr,watched_cnt
0,59879000,4
1,59895000,1
2,59900000,8
3,59921000,2
4,59930000,12
...,...,...
327,67117000,10
328,67129000,1
329,67140000,3
330,67148000,11


In [32]:
df8 = df6.merge(df7, on = 'subsr', how = 'left')
df8 = df8[df8['watched_cnt'] != 0].reset_index(drop = True)
df8

,subsr,series_nm,vod_id,watched,watched_all,watched_cnt
0,59879000,소방서 옆 경찰서,2092,1,11,4
1,59879000,신성한 이혼,1190,3,12,4
2,59895000,금이야 옥이야,784,1,83,1
3,59900000,2022 역사저널 그날,572,1,3,8
4,59900000,그것이알고싶다,297,3,21,8
...,...,...,...,...,...,...
1495,67140000,경남 통영 2부,4396,1,1,3
1496,67140000,밀수,2865,1,2,3
1497,67140000,잠자는 숲속의 공주,689,1,1,3
1498,67148000,타요의 씽씽극장 동요2,880,11,13,11


In [33]:
import numpy as np
def scoring(df):
  N = df['watched_all']
  L = df['watched_cnt']
  n = df['watched']
  lam = np.log(2) / 2
  w1 = 1 - np.exp(-1 * lam * N)
  data = pd.DataFrame([N, L]).T
  def custom_weight(data):
    if data['watched_cnt'] < data['watched_all']:
      return data['watched_all'] / data['watched_cnt']
    else:
      return 1

  w2 = data.apply(custom_weight, axis = 1)

  score = (n / N) * w1 * w2

  return score

In [34]:
df8['score'] = scoring(df8)
df8

,subsr,series_nm,vod_id,watched,watched_all,watched_cnt,score
0,59879000,소방서 옆 경찰서,2092,1,11,4,0.244476
1,59879000,신성한 이혼,1190,3,12,4,0.738281
2,59895000,금이야 옥이야,784,1,83,1,1.000000
3,59900000,2022 역사저널 그날,572,1,3,8,0.215482
4,59900000,그것이알고싶다,297,3,21,8,0.374741
...,...,...,...,...,...,...,...
1495,67140000,경남 통영 2부,4396,1,1,3,0.292893
1496,67140000,밀수,2865,1,2,3,0.250000
1497,67140000,잠자는 숲속의 공주,689,1,1,3,0.292893
1498,67148000,타요의 씽씽극장 동요2,880,11,13,11,0.988951


In [35]:
df3.head(3)

,subsr,asset_nm,vod_id,use_tms,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm,rename,watched,series_nm
0,59879000,소방서 옆 경찰서 05회,2092,2.4,TV드라마,기타,"김래원,손호준,공승연",69,소방서옆경찰서,0,소방서 옆 경찰서
1,59879000,소방서 옆 경찰서 06회,2092,22.1,TV드라마,기타,"김래원,손호준,공승연",59,소방서옆경찰서,1,소방서 옆 경찰서
2,59879000,신성한 이혼 01회,1190,12.6,TV드라마,기타,"조승우,한혜진,김성균,정문성",64,신성한이혼,0,신성한 이혼


In [36]:
vod_paper_score = df8[['subsr', 'series_nm', 'vod_id', 'score']]
vod_paper_score = vod_paper_score.merge(df3[['series_nm', 'vod_id', 'ct_cl', 'genre_of_ct_cl']].drop_duplicates(), on = ['series_nm', 'vod_id'], how = 'left')
vod_paper_score = vod_paper_score.sort_values(by = 'subsr').reset_index(drop = True)
vod_paper_score['category'] = vod_paper_score['ct_cl'].apply(lambda x : x if x in ['영화', '키즈'] else 'TV프로그램')

vod_paper_score

,subsr,series_nm,vod_id,score,ct_cl,genre_of_ct_cl,category
0,59879000,소방서 옆 경찰서,2092,0.244476,TV드라마,기타,TV프로그램
1,59879000,신성한 이혼,1190,0.738281,TV드라마,기타,TV프로그램
2,59895000,금이야 옥이야,784,1.000000,TV드라마,기타,TV프로그램
3,59900000,2022 역사저널 그날,572,0.215482,TV 시사/교양,기타,TV프로그램
4,59900000,그것이알고싶다,297,0.374741,TV 시사/교양,기타,TV프로그램
...,...,...,...,...,...,...,...
1496,67140000,경남 통영 2부,4396,0.292893,우리동네,연예/오락,TV프로그램
1497,67140000,밀수,2865,0.250000,영화,액션/어드벤쳐,영화
1498,67140000,잠자는 숲속의 공주,689,0.292893,키즈,기타,키즈
1499,67148000,타요의 씽씽극장 동요2,880,0.988951,키즈,기타,키즈


In [37]:
vod_paper_score.to_csv('../data/전처리data/vod_paper_score.csv', index = 0)